In [ ]:
# Playground
# Producer
from google.cloud import pubsub_v1

publisher = pubsub_v1.PublisherClient()
topic_path = publisher.topic_path("my_project_name", "natality")

data = "Hello World!".encode('utf-8')
publisher.publish(topic_path, data=data)

# Consumer
import time
from google.cloud import pubsub_v1

subscriber = pubsub_v1.SubscriberClient()
subscription_path = subscriber.subscription_path("my_project_name", "dsp")

def callback(message):
    print(message.data)
    message.ack()

subscriber.subscribe(subscription_path, callback=callback)

while True:
    time.sleep(10)

In [ ]:
import apache_beam as beam
import argparse
from apache_beam.options.pipeline_options import PipelineOptions
from apache_beam.io.gcp.bigquery import parse_table_schema_from_json
import json

class ApplyDoFn(beam.DoFn):

    def __init__(self):
        self._model = None
        from google.cloud import storage
        import pandas as pd
        import pickle as pkl
        import json as js
        self._storage = storage
        self._pkl = pkl
        self._pd = pd
        self._json = js
     
    def process(self, element):
        if self._model is None:
            bucket = self._storage.Client().get_bucket('model_store')
            blob = bucket.get_blob('natality/sklearn-linear')
            self._model =self._pkl.loads(blob.download_as_string())
        
        element = self._json.loads(element.decode('utf-8'))
        new_x = self._pd.DataFrame.from_dict(element, 
                            orient = "index").transpose().fillna(0)   
        weight = self._model.predict(new_x.iloc[:,1:8])[0]
        return [ { 'guid': element['guid'], 'weight': weight, 
                                   'time': str(element['time']) } ]

In [ ]:
class PublishDoFn(beam.DoFn):
    
    def __init__(self):
        from google.cloud import datastore       
        self._ds = datastore
    
    def process(self, element):
        client = self._ds.Client()
        key = client.key('natality-guid', element['guid'])
        entity = self._ds.Entity(key)
        entity['weight'] = element['weight']         
        entity['time'] = element['time']
        client.put(entity)

In [ ]:
# set up pipeline parameters 
parser = argparse.ArgumentParser()
known_args, pipeline_args = parser.parse_known_args(None)
pipeline_options = PipelineOptions(pipeline_args)

# define the topics 
topic = "projects/{project}/topics/{topic}"
topic = topic.format(project="my_project_name", topic="natality")

# define the pipeline steps 
p = beam.Pipeline(options=pipeline_options)
lines = p | 'Read PubSub' >> beam.io.ReadFromPubSub(topic=topic)
scored = lines | 'apply' >> beam.ParDo(ApplyDoFn())
scored | 'Create entities' >> beam.ParDo(PublishDoFn())

# run the pipeline 
result = p.run()
result.wait_until_finish()

In [ ]:
import json
from google.cloud import pubsub_v1
import time 

data = json.dumps({'year': 2001, 'plurality': 1, 
     'apgar_5min': 99, 'mother_age': 33, 
     'father_age': 40, 'gestation_weeks': 38, 'ever_born': 8, 
     'mother_married': 1, 'weight': 6.8122838958, 
     'time': str(time.time()), 
     'guid': 'b281c5e8-85b2-4cbd-a2d8-e501ca816363'}
).encode('utf-8') 

publisher = pubsub_v1.PublisherClient()
topic_path = publisher.topic_path("my_project_name", "natality")
publisher.publish(topic_path, data=data)